<a href="https://colab.research.google.com/github/mantuonweb/Google_Collab/blob/master/Agent_Resume_Matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain-openai langchain-community langchain-text-splitters langchain-core faiss-cpu python-dotenv pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1
ERROR: pip's dependency 

In [ ]:
import os
import shutil
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader
from dotenv import load_dotenv
from google.colab import userdata

# Load API key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
load_dotenv()
print("✓ Setup complete")
print("API Key:", os.environ.get("OPENAI_API_KEY", "Not set")[:20] + "...")

# Create folders if they don't exist
os.makedirs("./resumes", exist_ok=True)
os.makedirs("./resume_db", exist_ok=True)
print("✓ Folders created: ./resumes and ./resume_db")


✓ Setup complete
API Key: sk-proj-2D_k1B8OV3MW...
✓ Folders created: ./resumes and ./resume_db


In [ ]:
import shutil
import os

# Remove corrupted database
if os.path.exists("./resume_db"):
    shutil.rmtree("./resume_db")
    print("✓ Cleaned up old database")

# Recreate folder
os.makedirs("./resume_db", exist_ok=True)
print("✓ Ready for fresh start")


✓ Cleaned up old database
✓ Ready for fresh start


In [ ]:
import shutil
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader
from dotenv import load_dotenv
from google.colab import userdata

# Initialize embeddings globally
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

def ingest_resumes():
    """Load resumes from ./resumes folder and add to vector database"""
    print("📥 Ingesting resumes...")

    # Load text files
    txt_loader = DirectoryLoader("./resumes", glob="**/*.txt", loader_cls=TextLoader)
    txt_docs = txt_loader.load()

    # Load PDF files
    pdf_loader = DirectoryLoader("./resumes", glob="**/*.pdf", loader_cls=PyPDFLoader)
    pdf_docs = pdf_loader.load()

    all_docs = txt_docs + pdf_docs

    if not all_docs:
        print("❌ No resumes found in ./resumes folder")
        return

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(all_docs)

    # Check if FAISS index file exists (not just folder)
    db_file_exists = os.path.exists("./resume_db/index.faiss")

    if db_file_exists:
        # Load existing and add new documents
        vectorstore = FAISS.load_local("./resume_db", embeddings, allow_dangerous_deserialization=True)
        vectorstore.add_documents(chunks)
        print(f"✓ Added {len(chunks)} chunks from {len(all_docs)} resumes")
    else:
        # Create new vector store
        vectorstore = FAISS.from_documents(chunks, embeddings)
        print(f"✓ Created new database with {len(chunks)} chunks from {len(all_docs)} resumes")

    vectorstore.save_local("./resume_db")
    print("✓ Database saved successfully")


def list_resumes():
    """List all resumes stored in vector database"""
    print("📋 Listing resumes...")

    if not os.path.exists("./resume_db/index.faiss"):
        print("❌ No database found. Please ingest resumes first.")
        return

    vectorstore = FAISS.load_local("./resume_db", embeddings, allow_dangerous_deserialization=True)

    # Get all documents
    all_docs = vectorstore.docstore._dict

    # Extract unique sources
    sources = set()
    for doc in all_docs.values():
        if hasattr(doc, 'metadata') and 'source' in doc.metadata:
            sources.add(os.path.basename(doc.metadata['source']))

    print(f"\n✓ Found {len(sources)} resumes in database:")
    for i, source in enumerate(sorted(sources), 1):
        print(f"  {i}. {source}")


def search_resumes(skills):
    """Search resumes by skills and return best matches"""
    print(f"🔍 Searching for candidates with skills: {skills}")

    if not os.path.exists("./resume_db/index.faiss"):
        print("❌ No database found. Please ingest resumes first.")
        return

    vectorstore = FAISS.load_local("./resume_db", embeddings, allow_dangerous_deserialization=True)

    # Search for relevant resume chunks
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    docs = retriever.invoke(skills)

    # Create context from retrieved documents
    context = "\n\n".join([f"Resume {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)])

    # Create prompt for LLM
    prompt = f"""You are a recruiter assistant. Based on the following resume excerpts, identify and rank the best candidates for the required skills.\n\nRequired Skills: {skills}\n\nResume Excerpts:\n{context}\n\nPlease provide a quick summary for the top 3 best matching candidates. For each candidate, include their relevant skills, why they are a good fit, and a matching percentage. The response should be concise.\n\nAnswer:"""

    # Get LLM response
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    response = llm.invoke(prompt)

    print("\n" + "="*60)
    print("🎯 SEARCH RESULTS")
    print("="*60)
    print(response.content)
    print("="*60)

    return response.content


def clear_resumes():
    """Clear all resumes from vector database"""
    print("🗑️  Clearing resume database...")

    if os.path.exists("./resume_db"):
        shutil.rmtree("./resume_db")
        print("✓ Database cleared successfully")
    else:
        print("❌ No database found")

print("✓ Agent functions loaded successfully")

✓ Agent functions loaded successfully


In [ ]:
def generate_resume(data):
    """Generate a text resume from data dictionary"""
    resume = []

    # Header
    resume.append(data['name'].upper())
    resume.append(f"{data['email']} | {data['phone']} | {data['location']}")
    resume.append("")

    # Skills
    resume.append("SKILLS")
    resume.append(", ".join(data['skills']))
    resume.append("")

    # Experience
    resume.append("EXPERIENCE")
    for exp in data['experiences']:
        resume.append(f"{exp['title']} | {exp['company']} | {exp['duration']}")
        for resp in exp['responsibilities']:
            resume.append(f"- {resp}")
        resume.append("")

    # Education
    resume.append("EDUCATION")
    edu = data['education']
    resume.append(f"{edu['degree']} | {edu['institution']} | {edu['year']}")

    return "\n".join(resume)


def save_resume(data, filepath):
    """Save resume to file"""
    with open(filepath, 'w') as f:
        f.write(generate_resume(data))


# Example usage
if __name__ == "__main__":
    resume_data = {
        "name": "Mantu Nigam",
        "email": "mantu.nigam@email.com",
        "phone": "+91-9876543210",
        "location": "Bangalore",
        "skills": ["Python", "React", "Angular", "Nest", "Html", "CSS", "Google Cloud", "Docker"],
        "experiences": [
            {
                "title": "Senior AI Engineer",
                "company": "TechCorp",
                "duration": "2021-Present",
                "responsibilities": [
                    "Built Full stack applications with Angular and Nest",
                    "Developed Mobile App Using Material UI"
                ]
            },
            {
                "title": "Software Engineer",
                "company": "TCS",
                "duration": "2022-2025",
                "responsibilities": [
                    "Created ML models and REST APIs with Python"
                ]
            }
        ],
        "education": {
            "degree": "MCA",
            "institution": "IPU Delhi",
            "year": "2011"
        }
    }

    # Generate and print
    print(generate_resume(resume_data))

    # Save to file
    save_resume(resume_data, "resumes/mantu_nigam_resume.txt")


MANTU NIGAM
mantu.nigam@email.com | +91-9876543210 | Bangalore

SKILLS
Python, React, Angular, Nest, Html, CSS, Google Cloud, Docker

EXPERIENCE
Senior AI Engineer | TechCorp | 2021-Present
- Built Full stack applications with Angular and Nest
- Developed Mobile App Using Material UI

Software Engineer | TCS | 2022-2025
- Created ML models and REST APIs with Python

EDUCATION
MCA | IPU Delhi | 2011


In [ ]:
def generate_resume(data):
    """Generate a text resume from data dictionary"""
    resume = []

    # Header
    resume.append(data['name'].upper())
    resume.append(f"{data['email']} | {data['phone']} | {data['location']}")
    resume.append("")

    # Skills
    resume.append("SKILLS")
    resume.append(", ".join(data['skills']))
    resume.append("")

    # Experience
    resume.append("EXPERIENCE")
    for exp in data['experiences']:
        resume.append(f"{exp['title']} | {exp['company']} | {exp['duration']}")
        for resp in exp['responsibilities']:
            resume.append(f"- {resp}")
        resume.append("")

    # Education
    resume.append("EDUCATION")
    edu = data['education']
    resume.append(f"{edu['degree']} | {edu['institution']} | {edu['year']}")

    return "\n".join(resume)


def save_resume(data, filepath):
    """Save resume to file"""
    with open(filepath, 'w') as f:
        f.write(generate_resume(data))


# Example usage
if __name__ == "__main__":
    resume_data = {
        "name": "Vinod Malik",
        "email": "vinod.malik@email.com",
        "phone": "+91-9876543210",
        "location": "Bangalore",
        "skills": ["Python", "LangChain", "VectorDB", "Google Cloud", "Docker"],
        "experiences": [
            {
                "title": "Senior AI Engineer",
                "company": "TechCorp",
                "duration": "2021-Present",
                "responsibilities": [
                    "Built Full stack Gen AI App",
                    "Developed Mobile App Using Material UI"
                ]
            },
            {
                "title": "Software Engineer",
                "company": "TCS",
                "duration": "2022-2025",
                "responsibilities": [
                    "Created ML models and REST APIs with Python"
                ]
            }
        ],
        "education": {
            "degree": "MCA",
            "institution": "IPU Delhi",
            "year": "2011"
        }
    }

    # Generate and print
    print(generate_resume(resume_data))

    # Save to file
    save_resume(resume_data, "resumes/vinod_malik_resume.txt")


VINOD MALIK
vinod.malik@email.com | +91-9876543210 | Bangalore

SKILLS
Python, LangChain, VectorDB, Google Cloud, Docker

EXPERIENCE
Senior AI Engineer | TechCorp | 2021-Present
- Built Full stack Gen AI App
- Developed Mobile App Using Material UI

Software Engineer | TCS | 2022-2025
- Created ML models and REST APIs with Python

EDUCATION
MCA | IPU Delhi | 2011


In [ ]:
# Add resumes to database
ingest_resumes()


📥 Ingesting resumes...
✓ Created new database with 2 chunks from 2 resumes
✓ Database saved successfully


In [ ]:
# Search for candidates with specific skills
skills = "front end, angular, react, microservice using nestjs"  # Change this to your required skills
search_resumes(skills)

🔍 Searching for candidates with skills: front end, angular, react, microservice using nestjs

🎯 SEARCH RESULTS
### Top Candidates Summary

**1. Mantu Nigam**  
- **Relevant Skills:** React, Angular, Nest, Full stack development  
- **Why They Are a Good Fit:** Mantu has direct experience building full stack applications using Angular and Nest, which aligns perfectly with the required skills. His background in both front-end and back-end technologies makes him a strong candidate for roles involving microservices.  
- **Matching Percentage:** 90%

**2. Vinod Malik**  
- **Relevant Skills:** (Limited relevant skills)  
- **Why They Are a Good Fit:** While Vinod has experience as a Senior AI Engineer, his resume does not mention any front-end technologies like Angular or React, nor does it indicate experience with microservices using Nest. His skills are more focused on AI and Python, making him less suitable for the required role.  
- **Matching Percentage:** 40%

### Summary
Mantu Nigam 

'### Top Candidates Summary\n\n**1. Mantu Nigam**  \n- **Relevant Skills:** React, Angular, Nest, Full stack development  \n- **Why They Are a Good Fit:** Mantu has direct experience building full stack applications using Angular and Nest, which aligns perfectly with the required skills. His background in both front-end and back-end technologies makes him a strong candidate for roles involving microservices.  \n- **Matching Percentage:** 90%\n\n**2. Vinod Malik**  \n- **Relevant Skills:** (Limited relevant skills)  \n- **Why They Are a Good Fit:** While Vinod has experience as a Senior AI Engineer, his resume does not mention any front-end technologies like Angular or React, nor does it indicate experience with microservices using Nest. His skills are more focused on AI and Python, making him less suitable for the required role.  \n- **Matching Percentage:** 40%\n\n### Summary\nMantu Nigam is the clear top candidate due to his relevant experience with both Angular and Nest, while Vinod

In [ ]:
# Search for candidates with specific skills
skills = "python, ML, Gen AI"  # Change this to your required skills
search_resumes(skills)

🔍 Searching for candidates with skills: python, ML, Gen AI

🎯 SEARCH RESULTS
### Top Candidates Summary

**1. Vinod Malik**  
- **Relevant Skills:** Python, ML, Gen AI  
- **Why a Good Fit:** Vinod has direct experience in building a full-stack Gen AI application and has created ML models using Python. His role as a Senior AI Engineer indicates a strong background in AI technologies.  
- **Matching Percentage:** 95%

**2. Mantu Nigam**  
- **Relevant Skills:** Python, ML  
- **Why a Good Fit:** Mantu has experience in creating ML models and REST APIs with Python. However, he lacks specific experience in Gen AI, which slightly lowers his fit compared to Vinod.  
- **Matching Percentage:** 85%

**3. (No third candidate)**  
- **Why No Third Candidate:** Only two candidates provided relevant experience and skills related to the required skills of Python, ML, and Gen AI. 

### Summary
Vinod Malik is the strongest candidate due to his direct experience with Gen AI, followed by Mantu Nigam, 

'### Top Candidates Summary\n\n**1. Vinod Malik**  \n- **Relevant Skills:** Python, ML, Gen AI  \n- **Why a Good Fit:** Vinod has direct experience in building a full-stack Gen AI application and has created ML models using Python. His role as a Senior AI Engineer indicates a strong background in AI technologies.  \n- **Matching Percentage:** 95%\n\n**2. Mantu Nigam**  \n- **Relevant Skills:** Python, ML  \n- **Why a Good Fit:** Mantu has experience in creating ML models and REST APIs with Python. However, he lacks specific experience in Gen AI, which slightly lowers his fit compared to Vinod.  \n- **Matching Percentage:** 85%\n\n**3. (No third candidate)**  \n- **Why No Third Candidate:** Only two candidates provided relevant experience and skills related to the required skills of Python, ML, and Gen AI. \n\n### Summary\nVinod Malik is the strongest candidate due to his direct experience with Gen AI, followed by Mantu Nigam, who has solid Python and ML experience but lacks Gen AI expo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')